<div style = "text-align: right"> Roger Lee </div>

<left>FINM 32000</left> 
<br>
<left>Spring 2024</left>

<h2><center>Homework # 7</center></h2>
<h3><center><font color = "blue">Solution By: Imtiaz Ali</font></center></h3>

In [1]:
import copy
import numpy as np
import pandas as pd
import scipy.optimize
from scipy.stats import norm
import statsmodels.api as sm
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')


In [2]:
%%time
####================================================
import sys

### Get Python and Jupyter Notebook version and information
# print('=' * 50)
# print(sys.executable)
print('=' * 50)
print(sys.version)
print('=' * 50)
print(sys.version_info)
print('=' * 50)
!python --version
print('=' * 50)
!jupyter --version
print('=' * 50)


3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=12, releaselevel='final', serial=0)
Python 3.8.12
Selected Jupyter core packages...
IPython          : 8.12.0
ipykernel        : 6.19.2
ipywidgets       : 8.0.4
jupyter_client   : 8.1.0
jupyter_core     : 5.3.0
jupyter_server   : 1.13.5
jupyterlab       : 3.3.2
nbclient         : 0.5.13
nbconvert        : 6.1.0
nbformat         : 5.7.0
notebook         : 6.5.4
qtconsole        : 5.4.2
traitlets        : 5.7.1
CPU times: total: 46.9 ms
Wall time: 2.13 s


## Problem 1


> The code in the $\text{ipynb}$ file should do Problem $1$a if you set $\text{algorithm } = \text{ 'value'}$.<br>
> It should do Problem $1$b if you set $\text{algorithm } = \text{ 'policy'}$.
>


In [3]:
%%time
####================================================
### Model Dynamics (GBM)
class GBM:

    def __init__(self, S0: float, r: float, sigma: float) -> None:
        self.S0 = S0
        self.r = r
        self.sigma = sigma
        

####================================================
### Contract
class Put:

    def __init__(self, K: float, T: float) -> None:
        self.K = K
        self.T = T
        


CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time
####================================================
### Pricing Engine
class MCengine:

    def __init__(self, M: float, N: float, seed: float = 0, algorithm: str = 'value') -> None:
        self.M = M     ### Number of paths
        self.N = N     ### Number of time periods
        self.rng = np.random.default_rng(seed=seed) 
        ### Seeding the random number generator with a specified number helps make the calculations reproducible
        self.algorithm = algorithm
    
    
    ####================================================
    ### For Policy optimization approach, problem 1b
    ### -> If b << 0 then this function essentially returns nearly 1 if X < a, or nearly 0 if X > a
    ### -> but with some smoothing of the discontinuity, using a sigmoid function, to help the optimizer
    def softExercise(self, X: np.ndarray, a: np.float64, b: np.float64) -> np.ndarray:
        return 1 / (1 + np.exp(-b * (X - a)))
    
    
    ### -> It should be the negative of the expression inside the max() on the homework sheet
    ### -> Want to do "maximization", so take the negative because we are calling "minimize" 
    def negofMCaverageOfExpectedPayouts(self, coefficients: np.ndarray, x: np.ndarray, 
                                        exercisePayoff: np.ndarray, continuationPayoff: np.ndarray) -> np.ndarray:
        ### Make probability of exercising
        p = self.softExercise(x, *coefficients)   ### Array of length M
        
        ### Using a minimizer as a solver so multiply by negative since we want to maximize it
        negMCavg = -np.mean(p * exercisePayoff + (1-p) * continuationPayoff)
        
        return negMCavg
    
    
    def price_americanPut_GBM(self, contract: Put, dynamics: GBM) -> Tuple[float]:
        ### Separate dynamics and contract class attributes to variables
        K, T = contract.__dict__.values()
        S0, r, sigma = dynamics.__dict__.values()
        
        ### Make timestep size
        M, N, _, _ = self.__dict__.values()
        dt = T / N
        
        ### Generates pseudo-random normals (Zt)
        Zt = self.rng.normal(loc=0.0, scale=1.0, size=(M, N))   ### IID random normals
        
        ### Generate GBM paths (M-by-N)
        ### Note: Need to make drift term of size M-by-N so [[1, 2,..., N], [1, 2,..., N],..., [1, 2,..., N]])
        tile_rep = np.tile(np.arange(1, N+1), (M, 1)) 
        paths = S0 * np.exp((r - 0.5 * sigma**2) * dt * tile_rep + sigma * np.sqrt(dt) * np.cumsum(Zt, axis=1))
        
        ### This is the payoff (cashflow) along each path, discounted to time nn (for nn=N,N-1,...)
        ### It corresponds to the far right-hand column in each page of the Excel worksheet
        ### I'm initializing it for time nn=N
        payoffDiscounted = np.maximum(0, K - paths[:, -1]) ### Y_tN = Excercise Payoff at time-T (M-by-1)
        
        ### You could make payoffDiscounted to be a matrix because it depends on nn.
        ### But I will just reuse a 1-dimensional array, by overwriting the time nn+1 entries at time nn.
        ### Backward Induction from N-1 to 1
        for nn in np.arange(N-1, 0, -1):
            continuationPayoffDiscounted = np.exp(-r * dt) * payoffDiscounted  ### e^(-r*dt)Y_tau_n+1 (M-by-1)
            ### This is the CONTINUATION payoff (cashflow) along each path, discounted to time nn (for nn=N-1,N-2,...)
            ### It corresponds to the blue column in each page of the Excel worksheet
            ### Note that payoffDiscounted comes from the previous iteration
            ### -- which was at time nn+1. So now we discount back to time nn.
            
            ### Markov State Variable, used to construct set of "B" basis funcs in the regression estimate of f_n
            X = paths[:, nn-1]             ### X_tn = Markov State Variable           (M-by-1)
            exerciseValue = K - X          ### Y_tn = PAYOFF[X(t_n)] for tn < T       (M-by-1)
            mask_ITM = (exerciseValue > 0) ### ITM Masks: Checking if X < K then True (M-by-1)
            
            if self.algorithm == 'value':
                ### Problem 1a: Value function (Longstaff-Schwartz) Approach ###
                
                ### Use quadrature function -> Y = beta0 * 1 + beta1 * X^1 + beta2 * X^2
                ### An (M-by-3) array containing the basis functions (Same ones as L7.9-7.10, and Excel)
                basisfunctions = np.stack((np.ones(M), X, X**2), axis=1)
                
                ### linear regression of discounted payoff against basisfunctions using ITM paths only
                model_regression = sm.OLS(continuationPayoffDiscounted[mask_ITM], basisfunctions[mask_ITM])
                regression_fit = model_regression.fit()
                
                ### This will be an array of 3 estimated "betas"
                coefficients = regression_fit.params  ### (beta0, beta1, beta2)
                
                ### This is similar to the Red column in Excel [f_est_n(X(t_n))]    
                # estimatedContinuationValue = basisfunctions @ coefficients         ### Array (M-by-1)
                estimatedContinuationValue = regression_fit.predict(basisfunctions)  ### Array (M-by-1)
                
                ### This is a (M-by-1) array of Booleans [1_{Y_tn >= f_est_n(X_tn)}]
                whichPathsToExercise = (exerciseValue >= np.maximum(estimatedContinuationValue, 0))
                
            elif self.algorithm == 'policy':
                ### Problem 1b: Policy optimization approach to Reinforcement learning ###
                
                ### Chose Nelder-Mead optimizer because it's generating reasonable results with minimal coding effort
                ### But gradient methods, done properly, usually run faster
                (a_opt, b_opt) = scipy.optimize.minimize(fun=self.negofMCaverageOfExpectedPayouts, 
                                                         x0=(0, 0), 
                                                         args=(X, exerciseValue, continuationPayoffDiscounted), 
                                                         method='Nelder-Mead').x
                
                ### Using the right-hand side of the last equation on the homework sheet
                ### This obtains the hard exercise decision from the optimized soft exercise function
                ### It should be a length-M array of Booleans (as it was in the "value" approach.
                ### But here it comes from the softExercise function)
                ### 1_(softExercise(X,a_opt,b_opt)>=0.5)*1_(exerciseValue>0) = 1 then exercise at time t_n
                ### 1_(softExercise(X,a_opt,b_opt)<0.5)*1_(exerciseValue>0) = 0  then continue to hold at time t_n
                mask_value_1 = (self.softExercise(X, a_opt, b_opt) >= 0.5)
                mask_value_2 = (exerciseValue > 0)                     ### Same as mask_ITM
                whichPathsToExercise = (mask_value_1 & mask_value_2)   ### Bitwise operation, Keep Y_tn (M-by-1)
                
            else:
                raise ValueError('Unknown algorithm type')
            
            ### See the "discounted cashflow along path" column in Excel
            ### Keep either exercise (Y_tn) or continuation discounted payoff (e^(-r*dt)Y_tau_n+1) for each path
            whichPathsToNotExercise = np.logical_not(whichPathsToExercise)   ### Keep e^(-r*dt)Y_tau_n+1
            payoffDiscounted[whichPathsToExercise] = exerciseValue[whichPathsToExercise]
            payoffDiscounted[whichPathsToNotExercise] = continuationPayoffDiscounted[whichPathsToNotExercise]
            
        ### The time-0 calculation needs no regression
        continuationPayoffDiscounted = np.exp(-r * dt) * payoffDiscounted
        estimatedContinuationValue = np.mean(continuationPayoffDiscounted)
        putprice = max(K - S0, estimatedContinuationValue)   ### Final payoff

        return (putprice)
    

CPU times: total: 0 ns
Wall time: 0 ns


## Problem 1a


> Complete the coding of the provided $\text{ipynb}$ file which prices the Bermudan put option under $\text{GBM}$, with the same parameters as in the Excel worksheet from class (which has been posted on Canvas), using the Longstaff-Schwartz method.<br>
> &emsp; Report an estimated price, based on $10000$ paths.<br>
> &emsp; At each exercise date, do the regression using only the paths that are in-the-money (at that specific date $-$ so there may be different subsamples on different dates) $-$ unlike the Excel worksheet, which uses all of the paths.



In [5]:
%%time
####================================================
### Set Model Dynamics and Contract
hw7dynamics = GBM(S0=1, r=0.03, sigma=0.20)
hw7contract = Put(K=1.1, T=4)


CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time
####================================================
### Calucluate price for the Bermudan put option under GBM using the Longstaff-Schwartz method
algorithm = 'value'
column_name = ['M', 'Put_Price_Bermudan_GBM', 'Algorithm']
df_BermGBM_value = pd.DataFrame(columns=column_name)
for i in range(0, 10):
    M_std_err = 1000*(1+i)
    hw7MC = MCengine(M=M_std_err, N=4, seed=0, algorithm=algorithm)
    put_price_BermGBM = hw7MC.price_americanPut_GBM(hw7contract, hw7dynamics)
    np.set_printoptions(precision=4, suppress=True, formatter={'float_kind':'{:f}'.format})
    
    ### Store into dataframe
    df_BermGBM_value.loc[len(df_BermGBM_value)-1] = [M_std_err, put_price_BermGBM, algorithm]
    

CPU times: total: 62.5 ms
Wall time: 66 ms


In [7]:
%%time
####================================================
### Format dataframe and show results
df_BermGBM_value.index = df_BermGBM_value.index.format(formatter=lambda x: '{}'.format(x))
df_BermGBM_value.set_index('M', inplace=True)
df_BermGBM_value.index.name = 'M'
df_BermGBM_value


CPU times: total: 0 ns
Wall time: 0 ns


,Put_Price_Bermudan_GBM,Algorithm
M,,
1000,0.166735,value
2000,0.163411,value
3000,0.161025,value
4000,0.160038,value
5000,0.160209,value
6000,0.160200,value
7000,0.160176,value
8000,0.161135,value
9000,0.161460,value


## Problem 1b


> The Longstaff-Schwartz method can be regarded as an example of a $\textit{Reinforcement Learning}$ $(\text{RL})$ algorithm. It selects actions ("exercise" vs. "continue") to try to maximize an expected reward (option payoff) that depends on the transitions of a state variable (the underlying $X$).
> &emsp; In particular, Longstaff-Schwartz takes a $\textit{Value}$-function approach to solving the dynamic programming formulation of the Reinforcement Learning problem. It finds an estimate $\hat{f}_{n}$ (same notation as $\text{L}7$) of the $\textit{value function}$ for the continuation action, by using $\text{OLS}$ regression, of simulated continuation payoffs on the state variable. This estimated continuation value $\hat{f}_{n}$ is compared against the value function for the exercise action, which is just the payoff function (for example $\text{Payoff}(X) = K - X$ in the case of a put):<br><br>
$$\begin{align*}
&\text{If } \hat{f}_n\left(X_{t_{n}}\right) > \text{Payoff}\left(X_{t_{n}}\right) \text{ then continue to hold at time } t_{n} \\
&\text{If } \hat{f}_n\left(X_{t_{n}}\right) \leq \text{Payoff}\left(X_{t_{n}}\right) \text{ then exercise at time } t_{n}
\end{align*}$$
>
> Here we will consider a different approach to $\text{RL}$.<br>
> &emsp; In contrast to Value-function $\text{RL}$, another approach to Reinforcement Learning is the $\textit{Policy}$-based approach. Rather than trying to estimate the value function (for the continuation action), it tries to more directly optimize the time-$t_{n}$ policy function, let's denote it $\Phi$, which maps each $X$ to one of two outputs: $\{0, 1\}$, where $0$ denotes continuing to hold, while $1$ denotes stopping (exercising).
$$\begin{align*}
&\text{If } \Phi\left(X_{t_{n}}\right) = 0 \text{ then continue to hold at time } t_{n} \\
&\text{If } \Phi\left(X_{t_{n}}\right) = 1 \text{ then exercise at time } t_{n}
\end{align*}$$
>
> In the particular one-dimensional example of put pricing that we have been studying, we know what form the stopping policy function should take. In theory it should be an indicator function 
$$\Phi_{c_{n}}\left(X\right) = \textbf{1}_{X \leq c_{n}}$$
> with a parameter $c_{n}$ is a specific"critical" or "threshold" level of the stock price $X$. Below $c_{n}$ you should exercise, and above $c_{n}$ you should continue to hold the put. So, in principle, we could try to estimate the optimal threshold $c_{n}$ by choosing it to maximize the average, across all simulated paths, of the simulated payout resulting from the policy $\Phi_{c_{n}}$ at time $t_{n}$.
> &emsp; However, this optimization has some numerical dificulties, due to the piecewise constancy of this "hard stopping" decision function $\Phi$ which only has two outputs $\{0, 1\}$. So suppose that we optimize a smoother function, a "soft stoppin" decision function $\phi$ which produces outputs in the interval $\textit{between}$ $0$ and $1$. Let $\phi$ have two parameters $a, b$ (which may depend on the time slice $n$) and specifically let $\phi$ be<a name="Note-1"></a>[<sup>[1]</sup>](#Note-1) a $\textit{sigmoid}$ or $\textit{logistic}$ function of $b(X - a)$:
$$\begin{equation}
\phi_{a, b}\left(X\right) = \frac{1}{1+\exp{\left(-b\left(X-a\right)\right)}} \tag{*}
\end{equation}\label{eq:sig}$$
>
> For large negative $b$, the $\phi_{a, b}$ will behave similarly to $\Phi_{a}$, in that it's near $1$ for $X < a$ and near $0$ for $X > a$. But unlike the hard stopping decision function, the soft decision function $\phi$ is more optimizer-friendly, because it varies smoothly between $0$ and $1$. It can be interpreted as making the exercise decision randomly, with probability $\phi_{a, b}\left(X\right)$ of exercising, and probability $1 - \phi_{a, b}\left(X\right)$ of continuing to hold, conditional on $X$. At time $t_{n}$ the optimizer should optimize 
$$\max_{a,b}\left(\frac{1}{M}\sum^{M}_{m=1}\left(\phi_{a,b}\left(X^{m}_{t_{n}}\right)\times\left(K-X^{m}_{t_{n}}\right) + \left(1 - \phi_{a,b}\left(X^{m}_{t_{n}}\right)\right)\times\left(\text{Continuation payout on the $m$th path}\right)\right)\right)$$
>
> where $X^{m}$ denotes the $m$th simulated path. Then calculate payouts by converting this optimized soft stopping decision into a hard stopping decision by 
$$\Phi\left(X_{t_{n}}\right) = \textbf{1}_{\phi_{\hat{a}, \hat{b}}(X_{tn})>0.5}\times\textbf{1}_{\text{Payoff}(X_{tn})>0}$$
>
> where $\hat{a}$ and $\hat{b}$ denote the optimized parameter values. Multiplying by $\textbf{1}_{\text{Payoff}(X_{tn}) > 0}$ makes sure that you are not exercising $\text{OTM}$ options. It should not be needed if your $\phi$ has been trained correctly, but we include it as a precaution.
> &emsp; Implement this policy optimization approach, by completing the code in the $\text{ipynb}$ file. Most of the coding is already provided.

> &nbsp; <a name="Note-1"></a>[1](#Note-1) On this problem, which is simple in the sense that the exercise region in $X$-space is just a one-dimensional interval, a single sigmoid function $(\ref{eq:sig})$ is sufficient to approximate the optimal stopping policy.<br>
> &emsp; On harder problems, where the exercise region may be a complicated subset of a multidimensional $X$-space, the function $(\ref{eq:sig})$ can be upgraded to a $\textit{deep neural network}$.<br>
> &emsp; For instance see http://jmlr.org/papers/volume20/18-232/18-232.pdf



In [8]:
%%time
####================================================
### Set Model Dynamics and Contract
hw7dynamics = GBM(S0=1, r=0.03, sigma=0.20)
hw7contract = Put(K=1.1, T=4)


CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
%%time
####================================================
### Calucluate price for the Bermudan put option under GBM using the Policy optimization approach
algorithm = 'policy'
column_name = ['M', 'Put_Price_Bermudan_GBM', 'Algorithm']
df_BermGBM_policy = pd.DataFrame(columns=column_name)
for i in range(0, 10):
    M_std_err = 1000*(1+i)
    hw7MC = MCengine(M=M_std_err, N=4, seed=0, algorithm=algorithm)
    put_price_BermGBM = hw7MC.price_americanPut_GBM(hw7contract, hw7dynamics)
    np.set_printoptions(precision=4, suppress=True, formatter={'float_kind':'{:f}'.format})
    
    ### Store into dataframe
    df_BermGBM_policy.loc[len(df_BermGBM_policy)-1] = [M_std_err, put_price_BermGBM, algorithm]
    

CPU times: total: 2.58 s
Wall time: 2.57 s


In [10]:
%%time
####================================================
### Format dataframe and show results
df_BermGBM_policy.index = df_BermGBM_policy.index.format(formatter=lambda x: '{}'.format(x))
df_BermGBM_policy.set_index('M', inplace=True)
df_BermGBM_policy.index.name = 'M'
df_BermGBM_policy


CPU times: total: 0 ns
Wall time: 996 µs


,Put_Price_Bermudan_GBM,Algorithm
M,,
1000,0.170263,policy
2000,0.163954,policy
3000,0.162466,policy
4000,0.161024,policy
5000,0.161184,policy
6000,0.161250,policy
7000,0.160968,policy
8000,0.161947,policy
9000,0.162104,policy


## Problem 2


> Let $S$ be a non-dividend-paying stock that follows Geometric Brownian motion with $70\%$ volatility and time-$0$ price $S_{0} = 10$. The interest rate is $2\%$.
> &emsp; Find the time-$0$ price of an "$\text{EY}$" contract which pays at time $1$
$$\begin{cases} 
\left(S_{1} - 11\right)^{+} & S_{1} > 14 \text{ or } S_{0.5} > 12 \\
\left(S_{1} - 10\right)^{+} & \text{otherwise}
\end{cases}$$
> Use conditional Monte Carlo, conditioning on $S_{0.5}$. The formula for conditional expected discounted payoff will depend on whether $S_{0.5} > 12$ or $S_{0.5} \leq 12$.
> &emsp; See the $\text{ipynb}$ file.


>- We will need to apply Conditional monte Carlo to solve this
>- Let:
>>- $T_{0} = 0.5$
>>- $T_{1} = 1$
>>- $S_{0} = 10$
>>- $K_{\text{Thresh}} = 12$
>>- $K_{1} = 11$
>>- $K_{2} = 10$
>>- $K_{3} = 14$
>>- $r = 0.02$
>>- $R_{grow} = r$
>>- $\sigma = 0.70$

>- The conditional expected discounted payoff at time 0, given $(S_{T_{0}})$, can be expressed as:
$$\text{EY}\left(S_{T_{0}}\right) = e^{-rT_{0}} \cdot \mathbb{E}\left[ \text{Payoff}\left(S_{T_{1}}, S_{T_{0}}\right) | S_{T_{0}}\right]$$
>- Now, we have two cases based on $(S_{T_{0}})$:
>>- 1. If $\left(S_{T_{0}} > K_{\text{Thresh}}\right)$:
$$\text{EY}\left(S_{T_{0}} > K_{\text{Thresh}}\right) = e^{-rT_{0}} \cdot \mathbb{E}\left[\left(S_{T_{1}} - K_{1}\right)^{+} | S_{T_{0}} >K_{\text{Thresh}}\right]$$
>>- 2. If $\left(S_{T_{0}} \leq K_{\text{Thresh}}\right)$:<br>
$$\text{EY}\left(S_{T_{0}} \leq K_{\text{Thresh}}\right) = e^{-rT_{0}} \cdot \mathbb{E}\left[\left(S_{T_{1}} - K_{2}\right)^{+} | S_{T_{0}} \leq K_{\text{Thresh}}\right]$$
>- Note: We need to consider the additional condition when $\left(S_{T_{1}} > K_{3}\right)$ for case $2$

>- Our final form for the time-$0$ price is the following:
$$\begin{align*}
C^{BS}_{1} &= C^{BS}\left(S_{t}=S_{T_{0}}, t=T_{0}, K=K_{1}, T=T_{1}, R_{grow}, r, \sigma\right) \\
C^{BS}_{2} &= C^{BS}\left(S_{t}=S_{T_{0}}, t=T_{0}, K=K_{2}, T=T_{1}, R_{grow}, r, \sigma\right) \\
CB^{BS}_{2} &= \exp\left(-r\left(T_{1} - T_{0}\right)\right)\mathcal{N}\left(d_{2}\left(K=K_{3}\right)\right) \\
d_{2} &= \frac{\ln\left(\frac{S_{T_{0}}}{K_{3}}\right) + \left(R_{grow} - \frac{1}{2}\sigma^{2}\right)\left(T_{1}-T_{0}\right)}{{\sigma\sqrt{T_{1}-T_{0}}}} \\
V_{0} &= \exp\left(-rT_{0}\right)\left[\mathbb{1}\{S_{T_{0}} > K_{\text{Thresh}}\}C^{BS}_{1} + \mathbb{1}\{S_{T_{0}} \leq K_{\text{Thresh}}\}\left(C^{BS}_{2} - CB^{BS}_{2}\right)\right] \\
\end{align*}$$


In [11]:
%%time
####================================================
### Model Dynamics (GBM)
class GBM:

    def __init__(self, sigma: float, r: float, drift: float, S_t: float, t: float) -> None:
        self.sigma = sigma
        self.r = r
        self.drift = drift   #sometimes we denoted this as "rGrow"
        self.S_t = S_t
        self.t = t
    
    
####================================================
### Contract
class CallOption:

    def __init__(self, K: float, T: float) -> None:
        self.K = K
        self.T = T
    
    
class CallBinary:

    def __init__(self, K: float, T: float) -> None:
        self.K = K
        self.T = T
    
    
class EYcontract:

    def __init__(self, threshold0: float, K_case1: float, K_case2lower: float, K_case2upper: float, 
                 T0: float, T1: float) -> None:
        self.threshold0 = threshold0
        self.K_case1 = K_case1
        self.K_case2lower = K_case2lower
        self.K_case2upper = K_case2upper
        self.T0 = T0
        self.T1 = T1 
    
    

CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
%%time
####================================================
### Pricing Engine
class AnalyticEngine:

    def __init__(self):
        pass
    
    
    def BSpriceCall(self, dynamics: GBM, contract: CallOption) -> float:
        ### Separate dynamics and contract class attributes to variables
        K, T = contract.__dict__.values()
        sigma, r, drift, S_t, t = dynamics.__dict__.values()
        
        ### Make BS variables and call price
        timeRemaining = T - t
        F = S_t * np.exp(drift * timeRemaining)
        std = sigma * np.sqrt(timeRemaining)
        d1 = np.log(F / K) / std + std / 2
        d2 = d1 - std
        call_price = np.exp(-r * timeRemaining) * (F * norm.cdf(d1) - K * norm.cdf(d2))
        
        return call_price
    
    
    def BSpriceCallBinary(self, dynamics: GBM, contract: CallBinary) -> float:
        ### Separate dynamics and contract class attributes to variables
        K, T = contract.__dict__.values()
        sigma, r, drift, S_t, t = dynamics.__dict__.values()
        
        ### Make BS variables and binary call price
        timeRemaining = T - t
        F = S_t * np.exp(drift * timeRemaining)
        std = sigma * np.sqrt(timeRemaining)
        d1 = np.log(F / K) / std + std / 2
        d2 = d1 - std
        binary_call_price = np.exp(-r * timeRemaining) * norm.cdf(d2)
        
        return binary_call_price
    
    
####================================================
class MCengine:

    def __init__(self, M: float, seed: float = 0) -> None:
        self.M = M                                  ### How many simulations
        self.rng = np.random.default_rng(seed=seed) 
        ### Seeding the random number generator with a specified number helps make the calculations reproducible
    
    
    def price_EYcontract(self, contract: EYcontract, dynamics: GBM) -> Tuple[float]:
        ### Separate dynamics and contract class attributes to variables
        threshold0, K_case1, K_case2lower, K_case2upper, T0, T1 = contract.__dict__.values()
        sigma, r, drift, S_t, t = dynamics.__dict__.values()

        ### Make Call and Binary Call contract object
        embeddedCallcase1 = CallOption(K=K_case1, T=T1)         ### K = 11
        embeddedCallcase2 = CallOption(K=K_case2lower, T=T1)    ### K = 10
        embeddedBinarycase2 = CallBinary(K=K_case2upper, T=T1)  ### K = 14
        dynamicsConditional = copy.copy(dynamics)
        dynamicsConditional.t = T0
        
        ### For time-T0 from time-0 ###
        ### Generates pseudo-random normals (Zt) and Brownian motion
        ZT0 = self.rng.normal(loc=0.0, scale=1.0, size=self.M)   ### IID random normals
        WT0 = np.sqrt(T0) * ZT0
        
        ### Generate GBM paths
        ST0 = S_t * np.exp((drift - 0.5 * (sigma**2)) * T0 + sigma * WT0) 
        
        ## For time-T1 from time-T0 ###
        ### Generates pseudo-random normals (Zt) and Brownian motion
        ZT1 = self.rng.normal(loc=0.0, scale=1.0, size=self.M)   ### IID random normals
        WT1 = np.sqrt(T1-T0) * ZT1
        
        ### Generate GBM paths
        ST1 = ST0 * np.exp((drift - 0.5 * (sigma**2)) * (T1-T0) + sigma * WT1) 
        
        ### Calculate call prices (method 1, vectorized)
        def handle_cases_method1(St0: np.ndarray, Kt0: float) -> np.ndarray:
            ### Make time-T0 discounted call and binary call prices with correct strikes 
            dynamicsConditional.S_t = St0
            call_price_case1 = AnalyticEngine().BSpriceCall(dynamicsConditional, embeddedCallcase1)
            call_price_case2A = AnalyticEngine().BSpriceCall(dynamicsConditional, embeddedCallcase2)
            call_price_case2B = AnalyticEngine().BSpriceCallBinary(dynamicsConditional, embeddedBinarycase2)
            
            ### Apply conditions (S_T0 > 12)
            mask_case1 = (St0 > Kt0)
            price_case1 = call_price_case1 
            price_case2 = call_price_case2A - call_price_case2B
            cp_conditional = np.exp(-r * T0) * np.where(mask_case1, price_case1, price_case2)
            
            return cp_conditional
        
        ### Calculate call prices (method 2, vectorized, need 100 Million samples to get close to method 1)
        def handle_cases_method2(St0: np.ndarray, St1: np.ndarray, Kt0: float, Kt1: float, 
                                 Kt1_case1: float, Kt1_case2: float) -> np.ndarray:
            ### Apply conditions (S_T1 > 14 or S_T0 > 12)
            mask_case1 = ((St1 > Kt1) | (St0 > Kt0))
            cp_conditional = np.exp(-r * T1) * np.where(mask_case1, np.maximum(St1 - Kt1_case1, 0), 
                                                        np.maximum(St1 - Kt1_case2, 0))
            
            return cp_conditional
        
        ### Get discounted call prices
        # call_price = handle_cases_method2(ST0, ST1, threshold0, K_case2upper, K_case1, K_case2lower)
        call_price = handle_cases_method1(ST0, threshold0)
        
        ### Calculate final expected call price and standard error
        price = np.mean(call_price)
        standard_error = np.std(call_price, ddof=1) / np.sqrt(self.M)

        return (price, standard_error)
    

CPU times: total: 0 ns
Wall time: 0 ns


In [13]:
%%time
####================================================
### Set Model Dynamics and Contract
hw7p2dynamics = GBM(sigma=0.7, r=0.02, drift=0.02, S_t=10, t=0)
hw7p2contract = EYcontract(threshold0=12, K_case1=11, K_case2lower=10, K_case2upper=14, T0=0.5, T1=1.0)


CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time
####================================================
### 
column_name = ['M', 'Call_Price', 'Standard_Error']
df_call_price = pd.DataFrame(columns=column_name)
for i in range(0, 10):
    M_std_err = 10000*(1+i)
    hw7p2MC = MCengine(M=M_std_err, seed=0)
    (price, std_err) = hw7p2MC.price_EYcontract(hw7p2contract, hw7p2dynamics)
    np.set_printoptions(precision=4, suppress=True, formatter={'float_kind':'{:f}'.format})
    
    ### Store into dataframe
    df_call_price.loc[len(df_call_price)-1] = [M_std_err, price, std_err]
    

CPU times: total: 203 ms
Wall time: 204 ms


In [15]:
%%time
####================================================
### Format dataframe and show results
df_call_price.index = df_call_price.index.format(formatter=lambda x: '{}'.format(x))
df_call_price.set_index('M', inplace=True)
df_call_price.index.name = 'M'
df_call_price


CPU times: total: 0 ns
Wall time: 1.03 ms


,Call_Price,Standard_Error
M,,
10000.0,2.551356,0.035096
20000.0,2.545311,0.025070
30000.0,2.549096,0.020698
40000.0,2.559512,0.018231
50000.0,2.553671,0.016248
60000.0,2.549000,0.014794
70000.0,2.545784,0.013650
80000.0,2.545846,0.012807
90000.0,2.547804,0.012097
